In [6]:
import numpy as np

In [7]:
# useful matrices
Identity_22 = np.eye(2, dtype=np.complex128)
Pauli_x = np.array([[0, 1], [1, 0]], dtype=np.complex128)

# threshold
thr = 10**-9

In [213]:
def is_unitary(A):
    n = A.shape[0]
    if (A.shape != (n, n)):
        raise ValueError("Matrix is not square.")
    A = np.array(A)
    return np.allclose(np.eye(n), A @ A.conj().T)


def is_identity(A):
    n = A.shape[0]
    if (A.shape != (n, n)):
        raise ValueError("Matrix is not square.")
    return np.allclose(A, np.eye(n))


def elimination_matrix(a,b):
    # a, b allowed to be complex
    
    # impose theta real + positive {eq.10}
    theta = np.arctan(abs(b/a))
    
    # lambda is the negative arg() of a
    lamda = - np.angle(a)
    
    # {eq.12}
    mu = np.pi + np.angle(b)
    
    # {eq.7}
    U_special = np.array([ [np.exp(1j*lamda) * np.cos(theta), np.exp(1j*mu) * np.sin(theta)],
                           [-np.exp(-1j*mu) * np.sin(theta), np.exp(-1j*lamda) * np.cos(theta)] ])
    
    return U_special


def two_level_decomp(A):
    n = A.shape[0]
    decomp = []
    indices = []
    A_c = np.copy(A)

    for i in range(n-2):
        for j in range(n-1, i, -1):

            a = A_c[i,j-1]
            b = A_c[i,j]

            # --- need checks --- 
            # if A[i,j] = 0, nothing to do! Except in last row - need to check diagonal element is 1 
            if abs(A_c[i,j]) < thr:
                U_22 = Identity_22

                if j == i+1:
                    U_22 = np.array([[1 / a, 0], [0, a]])

            # if A[i,j-1] = 0, need to swap columns - again checking last row to ensure diagonal element is 1 
            elif abs(A_c[i,j-1]) < thr:
                U_22 = Pauli_x

                if j == i+1:
                    U_22 = np.array([[1 / b, 0], [0, b]])

            # Special unitary matrix
            else: 
                U_22 = elimination_matrix(a,b)

            # ----- U_22 found -----

            # multiply submatrix of A with U_22
            A_c[:,(j-1,j)] = A_c[:,(j-1,j)] @ U_22

            # If not the identity matrix - represents a gate! So should store
            if not is_identity(U_22):
                decomp.append(U_22.conj().T)
                indices.append(np.array([j-1,j]))


        # check for diagonal element equal to 1
        assert np.allclose(A_c[i,i],1.0)
    
    # lower right hand 2x2 matrix remaining after decomp
    lower_rh_matrix = A_c[n-2:n, n-2:n]
    
    # if not equal to I - is a non trivial gate
    if not is_identity(lower_rh_matrix):
        decomp.append(lower_rh_matrix)
        indices.append(np.array([n-2,n-1]))

    return decomp, indices


def gray_method(M):
    
    A = M.copy()
    n = A.shape[0]
#     A = np.array(A)
    # using bitwise_xor find Gray permutations
    permutations = []
    for i in range(n):
        permutations.append(i ^ (i // 2))
        
    # 
    A[:,:] = A[:,permutations]
    A[:,:] = A[permutations,:]
    
    decomp, indices = two_level_decomp(A)
    new_ind = []
    
    for pair in indices:
        new_ind.append(np.sort(np.take(permutations, pair, 0)))
        
        
    return decomp, new_ind
    

In [14]:
# test matrix
w = np.exp((2j / 3) * np.pi)

In [27]:
A = np.array([[1, 1, 1, 0], 
                  [1, w, w * w, 0],
                  [1, w * w, w, 0], 
                  [0, 0, 0, -1j*np.sqrt(3)]]) / np.sqrt(3)
two_level_decomp(A)

([array([[ 0.70710678-0.00000000e+00j,  0.70710678-8.65956056e-17j],
         [-0.70710678-8.65956056e-17j,  0.70710678-0.00000000e+00j]]),
  array([[ 0.57735027-0.00000000e+00j,  0.81649658-9.99919924e-17j],
         [-0.81649658-9.99919924e-17j,  0.57735027-0.00000000e+00j]]),
  array([[-7.07106781e-01+8.65956056e-17j, -5.14325540e-16-7.07106781e-01j],
         [ 5.14325540e-16-7.07106781e-01j, -7.07106781e-01-8.65956056e-17j]]),
  array([[-6.98727119e-16-1.j,  0.00000000e+00+0.j],
         [ 0.00000000e+00+0.j,  0.00000000e+00-1.j]])],
 [array([1, 2]), array([0, 1]), array([1, 2]), array([2, 3])])

In [42]:
A = np.array([[1, 1, 1, 0], 
                  [1, w, w * w, 0],
                  [1, w * w, w, 0], 
                  [0, 0, 0, -1j*np.sqrt(3)]]) / np.sqrt(3)
gray_method(A)

([array([[0.-0.j, 1.-0.j],
         [1.-0.j, 0.-0.j]]),
  array([[ 0.70710678-0.00000000e+00j,  0.70710678-8.65956056e-17j],
         [-0.70710678-8.65956056e-17j,  0.70710678-0.00000000e+00j]]),
  array([[ 0.57735027-0.00000000e+00j,  0.81649658-9.99919924e-17j],
         [-0.81649658-9.99919924e-17j,  0.57735027-0.00000000e+00j]]),
  array([[-7.07106781e-01+8.65956056e-17j, -5.14325540e-16-7.07106781e-01j],
         [ 5.14325540e-16-7.07106781e-01j, -7.07106781e-01-8.65956056e-17j]]),
  array([[ 0.00000000e+00+0.j,  0.00000000e+00-1.j],
         [-6.98727119e-16-1.j,  0.00000000e+00+0.j]])],
 [array([3, 2]), array([1, 3]), array([0, 1]), array([1, 3]), array([3, 2])])

In [93]:
from scipy.stats import unitary_group
nq = 2
A = unitary_group.rvs(2**nq)
decomp, indices = two_level_decomp(A)
decomp, indices

([array([[ 0.75541506+0.51274538j, -0.37381713-0.16340445j],
         [ 0.37381713-0.16340445j,  0.75541506-0.51274538j]]),
  array([[ 0.20280304-2.04422190e-01j,  0.95764424-1.17277596e-16j],
         [-0.95764424-1.17277596e-16j,  0.20280304+2.04422190e-01j]]),
  array([[-0.17155617+4.56851000e-01j,  0.87284342-1.06892491e-16j],
         [-0.87284342-1.06892491e-16j, -0.17155617-4.56851000e-01j]]),
  array([[ 0.58477683-0.00485665j, -0.76493597+0.26997302j],
         [ 0.76493597+0.26997302j,  0.58477683+0.00485665j]]),
  array([[-0.50507414+3.46480514e-03j,  0.863069  -1.05695469e-16j],
         [-0.863069  -1.05695469e-16j, -0.50507414-3.46480514e-03j]]),
  array([[ 0.33508076-0.59560499j,  0.19080792-0.70467576j],
         [ 0.62162325+0.3828317j , -0.65206134-0.20454932j]])],
 [array([2, 3]),
  array([1, 2]),
  array([0, 1]),
  array([2, 3]),
  array([1, 2]),
  array([2, 3])])

In [ ]:
|1001>  |0011> -> (use these qubits to control the gate, act U on those 1/2 qubits) 

In [214]:
from functools import reduce
def recompose_matrix(matrix_seq, location_seq, num_qubits):
    U_list = []
    for M, loc in zip(matrix_seq, location_seq):
        U = np.eye(2**num_qubits, dtype=np.complex128)
        for i,k in enumerate(sorted(loc)):
            for j,l in enumerate(sorted(loc)):
                U[k, l] = M[i,j] 

        U_list.append(U)

    U_list.reverse()

    A_approx = reduce(lambda a,b : a @ b, U_list)
    
    return A_approx
    

# Stress tests

In [215]:
A = np.array([[1, 1, 1, 0], 
              [1, w, w * w, 0],
              [1, w * w, w, 0], 
              [0, 0, 0, -1j*np.sqrt(3)]]) / np.sqrt(3)

B = unitary_group.rvs(2**2)

num_qubits_C = 8

C = unitary_group.rvs(2**num_qubits_C)


## Two-level-decomp Method

In [216]:
# Test on A
matrix_seq, location_seq = two_level_decomp(A)
A_approx = recompose_matrix(matrix_seq, location_seq, num_qubits=2)
np.allclose(A, A_approx)

True

In [217]:
# Test on B
matrix_seq, location_seq = two_level_decomp(B)
B_approx = recompose_matrix(matrix_seq, location_seq, num_qubits=2)
np.allclose(B, B_approx)

True

In [212]:
# Test on C
matrix_seq, location_seq = two_level_decomp(C)
C_approx = recompose_matrix(matrix_seq, location_seq, num_qubits=num_qubits_C)
np.allclose(C, C_approx)

True

## Gray's Method

In [218]:
# Test on A
matrix_seq, location_seq = gray_method(A)
A_approx = recompose_matrix(matrix_seq, location_seq, num_qubits=2)
np.allclose(A, A_approx)

True

In [220]:
# Test on B
matrix_seq, location_seq = gray_method(B)
B_approx = recompose_matrix(matrix_seq, location_seq, num_qubits=2)
np.allclose(B, B_approx)

# Test on C
matrix_seq, location_seq = gray_method(C)
C_approx = recompose_matrix(matrix_seq, location_seq, num_qubits=num_qubits_C)
np.allclose(C, C_approx)

KeyboardInterrupt: 